In [1]:
import pandas as pd
import os
import numpy as np
import requests
import datetime
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import datetime
import time
import json
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.ticker import MultipleLocator
import openmeteo_requests
import requests_cache
from retry_requests import retry
import hsfs
from pathlib import Path
from dotenv import load_dotenv
import hopsworks
import sys

root_dir = Path().resolve().parent
sys.path.append(str(root_dir))

from format_data import format_weather_data, format_price_data
from get_electricity_prices import get_data, get_todays_data
from get_weather_data import get_historical_weather, get_weather_forecast



In [2]:
load_dotenv()
hopsworks_api = os.getenv("HOPSWORKS_API_KEY")

os.environ["HOPSWORKS_API_KEY"] = hopsworks_api

project = hopsworks.login()
fs = project.get_feature_store() 
print(f"Connected to project: {project.name}")

2025-01-03 12:39:11,053 INFO: Initializing external client
2025-01-03 12:39:11,054 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-03 12:39:13,038 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1164446
Connected to project: oskaralf


In [3]:
forecast = get_weather_forecast("Luleå", "X", "X", 65.5841, 22.1547)
print(forecast)

Coordinates 65.58380889892578°N 22.155181884765625°E
Elevation 15.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
              time_start  temperature_2m  precipitation  snow_depth  \
0    2025-01-03T00:00:00      -16.476000            0.0        0.14   
1    2025-01-03T01:00:00      -16.625999            0.0        0.14   
2    2025-01-03T02:00:00      -16.226000            0.0        0.14   
3    2025-01-03T03:00:00      -16.175999            0.0        0.14   
4    2025-01-03T04:00:00      -16.275999            0.1        0.14   
..                   ...             ...            ...         ...   
163  2025-01-09T19:00:00       -8.000000            0.1        0.36   
164  2025-01-09T20:00:00       -8.050000            0.1        0.36   
165  2025-01-09T21:00:00       -8.050000            0.1        0.36   
166  2025-01-09T22:00:00       -8.100000            0.1        0.36   
167  2025-01-09T23:00:00       -8.150000            0.1        0.36   

     pressure_msl  cl

In [4]:
price_fg = fs.get_feature_group(
    name='electricity_price_data',
    version=1,
)
weather_fg = fs.get_feature_group(
    name='weather_data',
    version=1,
)

In [5]:
weather_fg.insert(forecast)

2025-01-03 12:39:15,757 INFO: 	5 expectation(s) included in expectation_suite.
Validation failed.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1164446/fs/1155149/fg/1393145


Uploading Dataframe: 100.00% |██████████| Rows 168/168 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: weather_data_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1164446/jobs/named/weather_data_1_offline_fg_materialization/executions


(Job('weather_data_1_offline_fg_materialization', 'SPARK'),
 {
   "success": false,
   "results": [
     {
       "success": false,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "precipitation",
           "min_value": -0.0,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 695303
         }
       },
       "result": {
         "observed_value": 0.0,
         "element_count": 168,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-01-03T11:39:15.000756Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_col

In [6]:
price = get_todays_data("SE4")
price['time_start'] = price['time_start'].str.replace(r'\+\d{2}:\d{2}$', '', regex=True)
print(price)
print(price.dtypes)

         date           time_start    price
0  2025-01-03  2025-01-03T00:00:00  0.48896
1  2025-01-03  2025-01-03T01:00:00  0.43602
2  2025-01-03  2025-01-03T02:00:00  0.40263
3  2025-01-03  2025-01-03T03:00:00  0.38227
4  2025-01-03  2025-01-03T04:00:00  0.34317
5  2025-01-03  2025-01-03T05:00:00  0.35209
6  2025-01-03  2025-01-03T06:00:00  0.43934
7  2025-01-03  2025-01-03T07:00:00  0.59623
8  2025-01-03  2025-01-03T08:00:00  0.92167
9  2025-01-03  2025-01-03T09:00:00  0.92830
10 2025-01-03  2025-01-03T10:00:00  0.86666
11 2025-01-03  2025-01-03T11:00:00  0.78925
12 2025-01-03  2025-01-03T12:00:00  0.62298
13 2025-01-03  2025-01-03T13:00:00  0.60560
14 2025-01-03  2025-01-03T14:00:00  0.61509
15 2025-01-03  2025-01-03T15:00:00  0.83579
16 2025-01-03  2025-01-03T16:00:00  1.14957
17 2025-01-03  2025-01-03T17:00:00  1.21132
18 2025-01-03  2025-01-03T18:00:00  1.24265
19 2025-01-03  2025-01-03T19:00:00  1.15277
20 2025-01-03  2025-01-03T20:00:00  0.95026
21 2025-01-03  2025-01-03T21:00:

In [7]:
price_fg.insert(price)

2025-01-03 12:39:28,223 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1164446/fs/1155149/fg/1393146


Uploading Dataframe: 100.00% |██████████| Rows 24/24 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: electricity_price_data_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1164446/jobs/named/electricity_price_data_1_offline_fg_materialization/executions


(Job('electricity_price_data_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "price",
           "min_value": -0.1,
           "max_value": 5000,
           "strict_min": true
         },
         "meta": {
           "expectationId": 695305
         }
       },
       "result": {
         "observed_value": 0.34317,
         "element_count": 24,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-01-03T11:39:28.000223Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expectations": 1,
     "succ